# Sentiment

In [1]:
# pip install h5py

In [2]:
# pip install keras

In [3]:
# pip install PyYAML

In [4]:
%%time
import keras # for sentitment model
import tensorflow as tf  # to resolve the loading issue with different version of keras
from nltk.tokenize import RegexpTokenizer
import pickle # load transformers: 'tfidf_vectorizer' and 'selector'
import re # for tokenizer
import dill # load language model
from collections import defaultdict # to make language model work

Using TensorFlow backend.


Wall time: 7.99 s


In [5]:
%%time
# load model
model = tf.keras.models.load_model('models/kera_model_dropout_nn.h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1280064   
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________
Wall time: 721 ms


In [6]:
%%time
# load the tfidf_vectorizer from disk
filename = 'models/tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

# load the selector from disk
filename = 'models/selector.sav'
selector = pickle.load(open(filename, 'rb'))

Wall time: 2.93 s


In [7]:
def keras_predict_sentiment(text):
    """
    expect: a string of text
    modify: vectorize the string with 'tfidf_vectorizer' and 'selector' transformers
    return: an int (1:'postive' ; 2:'negative')
    """
    text_list=[]
    text_list.append(text)
    raw_dtm = tfidf_vectorizer.transform(text_list)
    selected_dtm = selector.transform(raw_dtm).astype('float32')  
    pred_class = int(model.predict_classes(selected_dtm.toarray()))    
    return pred_class

# Text Prediction

In [8]:
%%time
# Load models
with open('models/bigram_model.p', 'rb') as file:
   bigram_model = dill.load(file)
with open('models/trigram_model.p', 'rb') as file:
   trigram_model = dill.load(file)
with open('models/fourgram_model.p', 'rb') as file:
   fourgram_model = dill.load(file)
with open('models/fivegram_model.p', 'rb') as file:
   fivegram_model = dill.load(file)

# define tokenizer to get words
tokenizer = RegexpTokenizer(r'\w+')

Wall time: 1min 12s


In [9]:
# function to predict the next word based on bigram model
def bigram_predict_next_word(word):
    '''
    word: a list of token
    '''
    if len(bigram_model[word[0]]) == 0:
        return None
    else:
        prob_list = bigram_model[word[0]].values()
        # find the max prob
        most_likely = max(prob_list)
        #print(most_likely)
        # predicted words
        pred_words = [word for word, prob in bigram_model[word[0]].items() if prob == most_likely]
#         pred_word = random.choice(pred_words)
    return pred_words[0]

In [10]:
# function to predict next word with trigram model
def trigram_predict_next_word(words):
    '''
    words: a list of token
    '''
    if len(trigram_model[words[0], words[1]]) == 0:
        last_word = words[-1]
        return bigram_predict_next_word(last_word)
    else:
        # get probabilities of next word
        prob_list = trigram_model[words[0], words[1]].values()
        # find the max prob
        most_likely = max(prob_list)
        # predicted words
        pred_words = [word for word, prob in trigram_model[words[0], words[1]].items() if prob == most_likely]
#         pred_word = random.choice(pred_words)
    return pred_words[0]

In [11]:
def fourgram_predict_next_word(words):
    '''
    words: a list of token
    '''
    if len(fourgram_model[words[0], words[1], words[2]]) == 0:
        last_two_words = words[-2:] 
        return trigram_predict_next_word(last_two_words)
    else:
        # get probabilities of next word
        prob_list = fourgram_model[words[0], words[1], words[2]].values()
        # find max prob
        most_likely = max(prob_list)
        # get the predicted word(s)
        pred_words = [word for word, prob in fourgram_model[words[0], words[1], words[2]].items() if prob == most_likely]
        pred_word = random.choice(pred_words)
    return pred_words[0]

In [12]:
# function to predict next word with 5-gram model
def fivegram_predict_next_word(words):
    '''
    words: a list of token
    '''
    if len(fivegram_model[words[0], words[1], words[2], words[3]]) == 0:
        last_three_words = words[-3:]
        return fourgram_predict_next_word(last_three_words)
    else:
        # get probabilities of next word
        prob_list = fivegram_model[words[0], words[1], words[2], words[3]].values()
        # find max prob
        most_likely = max(prob_list)
        # predicted words
        pred_words = [w for w, p in fivegram_model[words[0], words[1], words[2], words[3]].items() if p == most_likely]
#         pred_word = random.choice(pred_words)
    return pred_words[0]

In [13]:
def ngram_prediction(text):
    """
    expect: a raw string of text
    modify: tokenize the string and # check the length to decide to start with which model.
                1. if the string has 4 tokens or above, take the last four tokens as input to fivegram_model
                2. if the string has 3 tokens, take the list of tokens as input to fourgram_model
                3. if the string has 2 tokens, take the list of tokens as input to trigram_model
                4. if the string has 1 tokens, take the token as input to trigram_model
    return: predicted word
    """
    # tokenize the words
    user_tokens = tokenizer.tokenize(text)
    if len(user_tokens) >= 4:
        try:
            return fivegram_predict_next_word(user_tokens[-4:]) # take the last four tokens
        except:    
            return fivegram_predict_next_word(user_tokens) # take the last four tokens
    elif len(user_tokens) == 3:
        return fourgram_predict_next_word(user_tokens)
    elif len(user_tokens) == 2:
        return trigram_predict_next_word(user_tokens)
    elif len(user_tokens) == 1:
        return bigram_predict_next_word(user_tokens) 
   

In [15]:
def ngram_prediction(text):
    """
    expect: a raw string of text
    modify: tokenize the string and # check the length to decide to start with which model.
            1. if the string has 4 tokens or above, take the last four tokens as input to fivegram_model
            2. if the string has 3 tokens, take the list of tokens as input to fourgram_model
            3. if the string has 2 tokens, take the list of tokens as input to trigram_model
            4. if the string has 1 token, take the token as input to bigram_model
    return: predicted word
    """
    # tokenize the words
    user_tokens = tokenizer.tokenize(text)
    if len(user_tokens) >= 4:
        try:
            return fivegram_predict_next_word(user_tokens[-4:]) # take the last four tokens
        except:    
            return fivegram_predict_next_word(user_tokens) # take the four tokens
    elif len(user_tokens) == 3:
        return fourgram_predict_next_word(user_tokens)
    elif len(user_tokens) == 2:
        return trigram_predict_next_word(user_tokens)
    elif len(user_tokens) == 1:
        return bigram_predict_next_word(user_tokens) 

# Anvil

In [16]:
import anvil.server

anvil.server.connect("3V3UIWCX7QXD7PQE7SUKSGH7-PDHT24SNT663M2H7")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [17]:
@anvil.server.callable
# Put the function that will be used in anvil client server (web)
# This block of code should be running forever
# The @anvil.server.callable decorator can only take one function

def app_prediction(text):
    """
    expect: a string of text
    modify: send the string to two functions:
                1. ngram_prediction
                2. keras_predict_sentiment
    return: a tuple that contains a predicted word and a predicted class
    """
    ## Sentiment Prediction
    pred_word = ngram_prediction(text)
    
    ## Text Prediction
    pred_class = keras_predict_sentiment(text)
        
    return pred_word, pred_class

anvil.server.wait_forever()

KeyboardInterrupt: 